In [1]:
!pip install tensorflow transformers

In [2]:
!pip install datasets

In [3]:
!pip install huggingface_hub

In [4]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import classification_report
from huggingface_hub import notebook_login
from datasets import load_dataset

In [5]:
# Load the dataset
dataset = load_dataset("gretelai/symptom_to_diagnosis")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
# Convert train_dataset to DataFrame
df_train = pd.DataFrame(dataset["train"])
df_test = pd.DataFrame(dataset["test"])

df_train['input_text'] = df_train['input_text'].astype(str)
df_test['input_text'] = df_test['input_text'].astype(str)

In [7]:
df_train.head(2)

,output_text,input_text
0,cervical spondylosis,I've been having a lot of pain in my neck and ...
1,impetigo,I have a rash on my face that is getting worse...


Label encoding

In [8]:
# Encode labels using LabelEncoder
label_encoder = LabelEncoder()
df_train['encoded_labels'] = label_encoder.fit_transform(df_train['output_text'])
df_test['encoded_labels'] = label_encoder.transform(df_test['output_text'])

In [9]:
label_encoder.classes_

array(['allergy', 'arthritis', 'bronchial asthma', 'cervical spondylosis',
       'chicken pox', 'common cold', 'dengue', 'diabetes',
       'drug reaction', 'fungal infection',
       'gastroesophageal reflux disease', 'hypertension', 'impetigo',
       'jaundice', 'malaria', 'migraine', 'peptic ulcer disease',
       'pneumonia', 'psoriasis', 'typhoid', 'urinary tract infection',
       'varicose veins'], dtype=object)

In [10]:
len(label_encoder.classes_)

22

The data is split into training and validation sets

In [11]:
# Split the data into train and validation sets
train_df, val_df = train_test_split(df_train, test_size=0.1, stratify=df_train['output_text'])

Tokenization of input text is done using the BERT tokenizer

In [12]:
# Tokenize the text data and apply preprocessing
max_length = 150
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

tokenized_train = tokenizer(list(train_df['input_text']), padding=True, truncation=True, max_length=max_length, return_tensors="tf")
tokenized_val = tokenizer(list(val_df['input_text']), padding=True, truncation=True, max_length=max_length, return_tensors="tf")

In [13]:
train_df['input_text'][0]

"I've been having a lot of pain in my neck and back. I've also been having trouble with my balance and coordination. I've been coughing a lot and my limbs feel weak."

In [14]:
tokenizer(train_df['input_text'][0], padding=True, truncation=True, max_length=max_length, return_tensors="tf")

{'input_ids': <tf.Tensor: shape=(1, 43), dtype=int32, numpy=
array([[  101,   146,   112,  1396,  1151,  1515,   170,  1974,  1104,
         2489,  1107,  1139,  2455,  1105,  1171,   119,   146,   112,
         1396,  1145,  1151,  1515,  3819,  1114,  1139,  5233,  1105,
        14501,   119,   146,   112,  1396,  1151, 24992,   170,  1974,
         1105,  1139, 10765,  1631,  4780,   119,   102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 43), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 43), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
      dtype=int32)>}

In [15]:
print(tokenized_train['input_ids'][0])

tf.Tensor(
[  101   146   112  1396  1151  1515   170  1974  1104  2489  1107  1139
  2455  1105  1171   119   146  1145  1631 21441  1105  4780   119   102
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0], shape=(76,), dtype=int32)


Labels are converted to one-hot encoded format.


In [16]:
# Convert labels to one-hot encoded format
Y_train_encoded = tf.keras.utils.to_categorical(train_df['encoded_labels'], num_classes=len(label_encoder.classes_))
Y_val_encoded = tf.keras.utils.to_categorical(val_df['encoded_labels'], num_classes=len(label_encoder.classes_))

In [17]:
train_df['encoded_labels'][0]

3

In [18]:
tf.keras.utils.to_categorical(train_df['encoded_labels'][0], num_classes=len(label_encoder.classes_))

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.], dtype=float32)

Create TensorFlow datasets

In [19]:
# Create TensorFlow datasets
BATCH_SIZE = 64
train_dataset = tf.data.Dataset.from_tensor_slices((dict(tokenized_train), Y_train_encoded)).batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.from_tensor_slices((dict(tokenized_val), Y_val_encoded)).batch(BATCH_SIZE)

#3. Model Definition:



*   The code defines a sequence classification model using BERT-based transformer architecture (TFAutoModelForSequenceClassification).
*   The model is compiled with categorical crossentropy loss and Adam optimizer.

*   The number of output classes is determined by the unique labels in the dataset.





In [20]:
# Define the model
num_classes = len(label_encoder.classes_)
int2label = dict(enumerate(label_encoder.classes_))
label2int = {v: k for k, v in int2label.items()}

model = TFAutoModelForSequenceClassification.from_pretrained(
    "bert-base-cased",
    num_labels=num_classes,
    id2label=int2label,
    label2id=label2int,
    output_attentions=True
)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
!pip install --upgrade transformers

In [30]:
import tensorflow as tf
from tensorflow.keras import losses, optimizers, metrics
init_lr = 2e-5
# Use the get_trainable_variables() method to get trainable variables
optimizer = tf.optimizers.AdamW(learning_rate=init_lr)
model.compile(
    loss=losses.CategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    metrics=['accuracy']
)

#4. Model Training:

In [31]:
# Train the model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=20
)

Epoch 1/20


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
12/12 [==============================] - 129s 2s/step - loss: 3.1095 - accuracy: 0.0430 - val_loss: 3.0535 - val_accuracy: 0.0581
Epoch 2/20
12/12 [==============================] - 11s 902ms/step - loss: 2.9499 - accuracy: 0.1656 - val_loss: 2.8163 - val_accuracy: 0.2209
Epoch 3/20
12/12 [==============================] - 11s 913ms/step - loss: 2.6540 - accuracy: 0.3820 - val_loss: 2.4700 - val_accuracy: 0.5233
Epoch 4/20
12/12 [==============================] - 11s 917ms/step - loss: 2.3253 - accuracy: 0.5945 - val_loss: 2.1484 - val_accuracy: 0.6163
Epoch 5/20
12/12 [==============================] - 11s 925ms/step - loss: 1.9863 - accuracy: 0.7445 - val_loss: 1.8418 - val_accuracy: 0.7209
Epoch 6/20
12/12 [==============================] - 11s 922ms/step - loss: 1.6677 - accuracy: 0.8449 - val_loss: 1.5237 - val_accuracy: 0.7674
Epoch 7/20
12/12 

#5. Model Evaluation:

In [32]:
# Evaluate the model on the test set
tokenized_test = tokenizer(list(df_test['input_text']), padding=True, truncation=True, max_length=max_length, return_tensors="tf")
Y_test_encoded = tf.keras.utils.to_categorical(df_test['encoded_labels'], num_classes=num_classes)

test_dataset = tf.data.Dataset.from_tensor_slices((dict(tokenized_test), Y_test_encoded)).batch(BATCH_SIZE)

test_loss, test_accuracy = model.evaluate(test_dataset)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

4/4 [==============================] - 13s 226ms/step - loss: 0.2305 - accuracy: 0.9623
Test Loss: 0.23045825958251953, Test Accuracy: 0.9622641801834106


#6. Classification Report:

In [33]:
# Print the classification report for more detailed metrics
predictions = model.predict(test_dataset)
predicted_labels = tf.argmax(predictions.logits, axis=1)
true_labels = tf.argmax(tf.constant(Y_test_encoded), axis=1)

print(classification_report(true_labels, predicted_labels, target_names=label_encoder.classes_))

4/4 [==============================] - 12s 222ms/step
                                 precision    recall  f1-score   support

                        allergy       0.91      1.00      0.95        10
                      arthritis       1.00      1.00      1.00        10
               bronchial asthma       1.00      1.00      1.00        10
           cervical spondylosis       0.91      1.00      0.95        10
                    chicken pox       1.00      1.00      1.00        10
                    common cold       1.00      1.00      1.00        10
                         dengue       0.90      0.90      0.90        10
                       diabetes       1.00      0.90      0.95        10
                  drug reaction       0.80      1.00      0.89         8
               fungal infection       1.00      1.00      1.00         9
gastroesophageal reflux disease       1.00      0.90      0.95        10
                   hypertension       0.91      1.00      0.95       

#7. Hugging Face Hub Integration:

In [36]:
notebook_login()
model.push_to_hub("AlperenEvci/bert-symptom-diagnosis", use_auth_token="")
tokenizer.push_to_hub("AlperenEvci/bert-symptom-diagnosis", use_auth_token="")

/usr/local/lib/python3.10/dist-packages/transformers/modeling_tf_utils.py:3188: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tf_model.h5:   0%|          | 0.00/434M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:836: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


README.md:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/AlperenEvci/bert-symptom-diagnosis/commit/89ba84706db037660abdb89438c22fe45746dac0', commit_message='Upload tokenizer', commit_description='', oid='89ba84706db037660abdb89438c22fe45746dac0', pr_url=None, pr_revision=None, pr_num=None)